# Peephole LSTM Test & Performance Comparison (Speed & Memory)

* [Imports](#Importing-necessary-modules)
* [Load & Definition](#Loading-and-defining-modules)
    * [Autograd Functions](#Autograd-Functions)
    * [Module Classes](#Module-classes-(C++,-CUDA,-PyTorch))
* [Models](#Defining-models)
    * [Definition](#Definition)
    * [Instantiation](#Instantiation)
    * [Parameter Synchronization](#Parameter-Synchronization)
* [Fake Dataset](#Creating-a-fake-dataset)
* [Sanity Check](#Sanity-check:-output-comparison)
    * [Forward Outputs](#Forward-Outputs)
    * [Backward Gradients](#Backward-Gradients)
* [Forward Performance](#Forward-time-comparison)
* [+Backward Performance](#+Backward-time-comparison)

---

## Importing necessary modules
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [16]:
if 'initialized' not in globals():
    import torch
    from torch import nn
    from torch.utils.cpp_extension import load
    from torch.nn import functional as F
    from torch.utils.data import TensorDataset, DataLoader

    import math
    from collections import OrderedDict
    from time import sleep

    initialized = [False] * 7
    print(torch.__version__)

---

## Loading and defining modules
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

### Autograd Functions
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [17]:
if not initialized[0]:
    _ln_peephole_lstm_layer_cpp = load('ln_peephole_lstm_layer', ['./ln_peephole_lstm_layer.cpp'])
    _ln_peephole_lstm_layer_cuda = load('ln_peephole_lstm_layer_cuda', ['./ln_peephole_lstm_layer_cuda.cpp', './ln_peephole_lstm_layer_cuda_kernel.cu'])

    ########################################################################################################################

    class LNPeepholeLSTMFunctionCPP(torch.autograd.Function):
        @staticmethod
        def forward(ctx, input, weight_ih, weight_hh, weight_ch, bias,
                    gamma_f, gamma_i, gamma_g, gamma_o, gamma_cell, beta_cell,
                    hidden, cell,
                    epsilon, dropout_p,
                    dropout_output, training):

            outputs = _ln_peephole_lstm_layer_cpp.forward(input, weight_ih, weight_hh, weight_ch, bias,
                                                          gamma_f, gamma_i, gamma_g, gamma_o, gamma_cell, beta_cell,
                                                          hidden, cell,
                                                          epsilon, dropout_p,
                                                          dropout_output, training)

            out, new_h, new_cell = outputs[:3]

            variables = outputs[3:] + [weight_ih, weight_hh, weight_ch,
                                       gamma_f, gamma_i, gamma_g, gamma_o, gamma_cell]
            ctx.save_for_backward(*variables)

            return out, new_h, new_cell

        @staticmethod
        def backward(ctx, grad_output, grad_h, grad_cell):
            outputs = _ln_peephole_lstm_layer_cpp.backward(
                grad_output.contiguous(), grad_h.contiguous(), grad_cell.contiguous(), *ctx.saved_tensors)

            (d_input, d_weight_ih, d_weight_hh, d_weight_ch, d_bias,
             d_gamma_f, d_gamma_i, d_gamma_o, d_gamma_g, d_gamma_cell, d_beta_cell,
             d_hidden, d_cell) = outputs

            return (d_input, d_weight_ih, d_weight_hh, d_weight_ch, d_bias,
                    d_gamma_f, d_gamma_i, d_gamma_o, d_gamma_g, d_gamma_cell, d_beta_cell,
                    d_hidden, d_cell,
                    None, None,
                    None, None)
        
    ########################################################################################################################
    
    class LNPeepholeLSTMFunctionCUDA(torch.autograd.Function):
        @staticmethod
        def forward(ctx, input, weight_ih, weight_hh, weight_ch, bias,
                    gamma_f, gamma_i, gamma_g, gamma_o, gamma_cell, beta_cell,
                    hidden, cell,
                    epsilon, dropout_p,
                    dropout_output, training):

            outputs = _ln_peephole_lstm_layer_cuda.forward(input, weight_ih, weight_hh, weight_ch, bias,
                                                          gamma_f, gamma_i, gamma_g, gamma_o, gamma_cell, beta_cell,
                                                          hidden, cell,
                                                          epsilon, dropout_p,
                                                          dropout_output, training)

            out, new_h, new_cell = outputs[:3]

            variables = outputs[3:] + [weight_ih, weight_hh, weight_ch,
                                       gamma_f, gamma_i, gamma_g, gamma_o, gamma_cell]
            ctx.save_for_backward(*variables)

            return out, new_h, new_cell

        @staticmethod
        def backward(ctx, grad_output, grad_h, grad_cell):
            outputs = _ln_peephole_lstm_layer_cuda.backward(
                grad_output.contiguous(), grad_h.contiguous(), grad_cell.contiguous(), *ctx.saved_tensors)

            (d_input, d_weight_ih, d_weight_hh, d_weight_ch, d_bias,
             d_gamma_f, d_gamma_i, d_gamma_o, d_gamma_g, d_gamma_cell, d_beta_cell,
             d_hidden, d_cell) = outputs

            return (d_input, d_weight_ih, d_weight_hh, d_weight_ch, d_bias,
                    d_gamma_f, d_gamma_i, d_gamma_o, d_gamma_g, d_gamma_cell, d_beta_cell,
                    d_hidden, d_cell,
                    None, None,
                    None, None)
        
    initialized[0] = True

### Module classes (PyTorch, C++, CUDA)
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [18]:
if not initialized[1]:
    class LNPeepholeLSTMTorch(nn.Module):
        def __init__(self, input_size, hidden_size, batch_first=False, dropout=0., dropout_on_output=True, eps=1e-05):
            if not 0 <= dropout <= 1:
                raise ValueError(f"Invalid dropout value : {dropout} dropout must be in range [0, 1].")
            super(LNPeepholeLSTMTorch, self).__init__()

            self.input_size = input_size
            self.hidden_size = hidden_size
            self.batch_first = bool(batch_first)
            self.dropout = float(dropout)
            self.dropout_on_output = bool(dropout_on_output)
            self.eps = eps

            self.register_parameter('weight_ih', nn.Parameter(torch.empty(4 * hidden_size, input_size)))
            self.register_parameter('weight_hh', nn.Parameter(torch.empty(4 * hidden_size, hidden_size)))
            self.register_parameter('weight_ch', nn.Parameter(torch.empty(3 * hidden_size)))
            self.register_parameter('bias', nn.Parameter(torch.empty(4 * hidden_size)))

            self.register_parameter('gamma_f', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('gamma_i', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('gamma_g', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('gamma_o', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('gamma_cell', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('beta_cell', nn.Parameter(torch.empty(hidden_size)))

            self.reset_parameters()

        def reset_parameters(self):
            stdv = 1.0 / math.sqrt(self.input_size + 2 * self.hidden_size)
            self.weight_ih.data.uniform_(-stdv, +stdv)
            self.weight_hh.data.uniform_(-stdv, +stdv)
            self.weight_ch.data.uniform_(-stdv, +stdv)

            self.bias.data.zero_()
            self.bias.data[:self.hidden_size].fill_(1.)

            self.gamma_f.data.uniform_()
            self.gamma_i.data.uniform_()
            self.gamma_g.data.uniform_()
            self.gamma_o.data.uniform_()
            self.gamma_cell.data.uniform_()
            self.beta_cell.data.zero_()

        def forward(self, input, states):
            assert input.dim() == 3, "expected a 3 dimensional tensor as `input`, but te given tensor has {} dimension(s)".format(input.dim())
            assert len(states) == 2, "expected a (hidden, cell) pair as `states`, but the length of the given states is {}".format(len(states))
            if self.batch_first:
                input = input.transpose(0, 1).contiguous()
            assert states[0].size() == (input.size(1), self.hidden_size), "expected a hidden state tensor with dimensionality {}, but the given tensor has dimensionality []".format(list(states[0].size()), [input.size(1), self.hidden_size])
            assert states[1].size() == (input.size(1), self.hidden_size), "expected a cell state tensor with dimensionality {}, but the given tensor has dimensionality []".format(list(states[1].size()), [input.size(1), self.hidden_size])

            hidden, cell = states

            hidden_size = self.hidden_size
            hidden_size_2 = 2 * hidden_size
            hidden_size_3 = hidden_size_2 + hidden_size

            norm_shape = torch.Size((hidden_size,))

            outputs = input.new_empty((input.size(0), input.size(1), hidden_size))
            
            ih = input.matmul(self.weight_ih.t())

            weight_hc_h = torch.cat((self.weight_hh.t(),
                                     torch.cat((self.weight_ch[:hidden_size].diag(),
                                                self.weight_ch[hidden_size:hidden_size_2].diag(),
                                                self.weight_ch.new_zeros(hidden_size_2, hidden_size))).t()))
            weight_co = self.weight_ch[hidden_size_2:]
            
            gamma_fig = torch.stack((self.gamma_f, self.gamma_i, self.gamma_g))

            bias_fig = torch.stack(self.bias[:hidden_size_3].chunk(3, dim=0))
            bias_o = self.bias[hidden_size_3:]

            for i in range(input.size(0)):
                gates = torch.addmm(ih[i], torch.cat((hidden, cell), dim=1), weight_hc_h).view(-1, 4, hidden_size)
                gates_fig = gates[:, :3]


                gates_fig = F.layer_norm(gates_fig, norm_shape, eps=self.eps)
                gates_fig = torch.addcmul(bias_fig, gates_fig, gamma_fig)
                forget_input_gates = gates_fig[:, :2].sigmoid()
                candidate_cell = F.dropout(gates_fig[:, 2].tanh(), p=self.dropout, training=self.training)


                cell = F.layer_norm(torch.addcmul(forget_input_gates[:, 0] * cell,
                                                  forget_input_gates[:, 1], candidate_cell),
                                    norm_shape, self.gamma_cell, self.beta_cell, self.eps)

                output_gate = torch.addcmul(gates[:, 3], cell, weight_co)

                output_gate = F.layer_norm(output_gate, norm_shape, self.gamma_o, bias_o, self.eps).sigmoid()

                hidden = output_gate * cell.tanh()

                outputs[i] = hidden

            if self.dropout_on_output:
                outputs = F.dropout(outputs, p=self.dropout, training=self.training)
                
            if self.batch_first:
                outputs = outputs.transpose(0, 1).contiguous()

            return outputs, (hidden, cell)

        def __repr__(self):
            return f"LNPeepholeLSTMTorch(input_size={self.input_size}, hidden_size={self.hidden_size}, batch_first={self.batch_first}, dropout={self.dropout}, dropout_on_output={self.dropout_on_output}, eps={self.eps})"

    ########################################################################################################################

    class LNPeepholeLSTMCPP(nn.Module):
        def __init__(self, input_size, hidden_size, batch_first=False, dropout=0., dropout_on_output=True, eps=1e-05):
            if not 0 <= dropout <= 1:
                raise ValueError(f"Invalid dropout value : {dropout} dropout must be in range [0, 1].")

            super(LNPeepholeLSTMCPP, self).__init__()

            self.input_size = input_size
            self.hidden_size = hidden_size
            self.batch_first = bool(batch_first)
            self.dropout = float(dropout)
            self.dropout_on_output = bool(dropout_on_output)
            self.eps = eps

            self.register_parameter('weight_ih', nn.Parameter(torch.empty(4 * hidden_size, input_size)))
            self.register_parameter('weight_hh', nn.Parameter(torch.empty(4 * hidden_size, hidden_size)))
            self.register_parameter('weight_ch', nn.Parameter(torch.empty(3 * hidden_size)))
            self.register_parameter('bias', nn.Parameter(torch.empty(4 * hidden_size)))

            self.register_parameter('gamma_f', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('gamma_i', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('gamma_g', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('gamma_o', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('gamma_cell', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('beta_cell', nn.Parameter(torch.empty(hidden_size)))

            self.reset_parameters()

        def reset_parameters(self):
            stdv = 1.0 / math.sqrt(self.input_size + 2 * self.hidden_size)
            self.weight_ih.data.uniform_(-stdv, +stdv)
            self.weight_hh.data.uniform_(-stdv, +stdv)
            self.weight_ch.data.uniform_(-stdv, +stdv)

            self.bias.data.zero_()
            self.bias.data[:self.hidden_size].fill_(1.)

            self.gamma_f.data.uniform_()
            self.gamma_i.data.uniform_()
            self.gamma_g.data.uniform_()
            self.gamma_o.data.uniform_()
            self.gamma_cell.data.uniform_()
            self.beta_cell.data.zero_()

        def forward(self, input, state):
            if self.batch_first:
                input = input.transpose(0, 1).contiguous()

            output, new_h, new_cell = LNPeepholeLSTMFunctionCPP.apply(
                input, self.weight_ih, self.weight_hh, self.weight_ch, self.bias,
                self.gamma_f, self.gamma_i, self.gamma_g, self.gamma_o, self.gamma_cell, self.beta_cell,
                state[0], state[1],
                self.eps, self.dropout, self.dropout_on_output, self.training)
            
            if self.batch_first:
                output = output.transpose(0, 1).contiguous()

            return output, (new_h, new_cell)

        def __repr__(self):
            return f"LNPeepholeLSTMCPP(input_size={self.input_size}, hidden_size={self.hidden_size}, batch_first={self.batch_first}, dropout={self.dropout}, dropout_on_output={self.dropout_on_output}, eps={self.eps})"
    
    ########################################################################################################################
    
    class LNPeepholeLSTMCUDA(nn.Module):
        def __init__(self, input_size, hidden_size, batch_first=False, dropout=0., dropout_on_output=True, eps=1e-05):
            if not 0 <= dropout <= 1:
                raise ValueError(f"Invalid dropout value : {dropout} dropout must be in range [0, 1].")

            super(LNPeepholeLSTMCUDA, self).__init__()

            self.input_size = input_size
            self.hidden_size = hidden_size
            self.batch_first = bool(batch_first)
            self.dropout = float(dropout)
            self.dropout_on_output = bool(dropout_on_output)
            self.eps = eps

            self.register_parameter('weight_ih', nn.Parameter(torch.empty(4 * hidden_size, input_size)))
            self.register_parameter('weight_hh', nn.Parameter(torch.empty(4 * hidden_size, hidden_size)))
            self.register_parameter('weight_ch', nn.Parameter(torch.empty(3 * hidden_size)))
            self.register_parameter('bias', nn.Parameter(torch.empty(4 * hidden_size)))

            self.register_parameter('gamma_f', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('gamma_i', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('gamma_g', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('gamma_o', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('gamma_cell', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('beta_cell', nn.Parameter(torch.empty(hidden_size)))

            self.reset_parameters()

        def reset_parameters(self):
            stdv = 1.0 / math.sqrt(self.input_size + 2 * self.hidden_size)
            self.weight_ih.data.uniform_(-stdv, +stdv)
            self.weight_hh.data.uniform_(-stdv, +stdv)
            self.weight_ch.data.uniform_(-stdv, +stdv)

            self.bias.data.zero_()
            self.bias.data[:self.hidden_size].fill_(1.)

            self.gamma_f.data.uniform_()
            self.gamma_i.data.uniform_()
            self.gamma_g.data.uniform_()
            self.gamma_o.data.uniform_()
            self.gamma_cell.data.uniform_()
            self.beta_cell.data.zero_()

        def forward(self, input, state):
            if self.batch_first:
                input = input.transpose(0, 1).contiguous()

            output, new_h, new_cell = LNPeepholeLSTMFunctionCUDA.apply(
                input, self.weight_ih, self.weight_hh, self.weight_ch, self.bias,
                self.gamma_f, self.gamma_i, self.gamma_g, self.gamma_o, self.gamma_cell, self.beta_cell,
                state[0], state[1],
                self.eps, self.dropout, self.dropout_on_output, self.training)
            
            if self.batch_first:
                output = output.transpose(0, 1).contiguous()

            return output, (new_h, new_cell)

        def __repr__(self):
            return f"LNPeepholeLSTMCUDA(input_size={self.input_size}, hidden_size={self.hidden_size}, batch_first={self.batch_first}, dropout={self.dropout}, dropout_on_output={self.dropout_on_output}, eps={self.eps})"
    
    initialized[1] = True

---

## Defining models
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

### Definition
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [19]:
if not initialized[2]:
    class LNPeepholeTorch(nn.Module):
        def __init__(self, input_size, hidden_size, output_size, n_layers=1, dropout=0, dropout_on_output=True, eps=1e-05):
            super().__init__()
            assert isinstance(n_layers, int)
            assert n_layers > 0

            self.lstm0 = LNPeepholeLSTMTorch(input_size, hidden_size, True, dropout, dropout_on_output, eps)
            for n in range(1, n_layers):
                self.add_module("lstm{}".format(n), LNPeepholeLSTMTorch(hidden_size, hidden_size, True, dropout, dropout_on_output, eps))
            self.fc = nn.Linear(hidden_size, output_size)

            self.input_size = input_size
            self.hidden_size = hidden_size
            self.output_size = output_size
            self.n_layers = n_layers

        def forward(self, x, states):
            assert states[0].dim() == 3
            assert states[0].size(0) == self.n_layers
            assert states[0].size(1) == x.size(0)
            assert states[0].size(2) == self.hidden_size
            assert states[0].size() == states[1].size()

            new_hidden = torch.empty_like(states[0])
            new_cell = torch.empty_like(states[1])

            for n in range(self.n_layers):
                x, (new_hidden[n], new_cell[n]) = getattr(self, "lstm{}".format(n))(x, (states[0][n], states[1][n]))
            x = self.fc(x)

            return x, (new_hidden, new_cell)

    ########################################################################################################################

    class LNPeepholeCPP(nn.Module):
        def __init__(self, input_size, hidden_size, output_size, n_layers=1, dropout=0, dropout_on_output=True, eps=1e-05):
            super().__init__()
            assert isinstance(n_layers, int)
            assert n_layers > 0

            self.lstm0 = LNPeepholeLSTMCPP(input_size, hidden_size, True, dropout, dropout_on_output, eps)
            for n in range(1, n_layers):
                self.add_module("lstm{}".format(n), LNPeepholeLSTMCPP(hidden_size, hidden_size, True, dropout, dropout_on_output, eps))
            self.fc = nn.Linear(hidden_size, output_size)

            self.input_size = input_size
            self.hidden_size = hidden_size
            self.output_size = output_size
            self.n_layers = n_layers

        def forward(self, x, states):
            assert states[0].dim() == 3
            assert states[0].size(0) == self.n_layers
            assert states[0].size(1) == x.size(0)
            assert states[0].size(2) == self.hidden_size
            assert states[0].size() == states[1].size()

            new_hidden = torch.empty_like(states[0])
            new_cell = torch.empty_like(states[1])

            for n in range(self.n_layers):
                x, (new_hidden[n], new_cell[n]) = getattr(self, "lstm{}".format(n))(x, (states[0][n], states[1][n]))
            x = self.fc(x)

            return x, (new_hidden, new_cell)
    
    ########################################################################################################################
    
    class LNPeepholeCUDA(nn.Module):
        def __init__(self, input_size, hidden_size, output_size, n_layers=1, dropout=0, dropout_on_output=True, eps=1e-05):
            super().__init__()
            assert isinstance(n_layers, int)
            assert n_layers > 0

            self.lstm0 = LNPeepholeLSTMCUDA(input_size, hidden_size, True, dropout, dropout_on_output, eps)
            for n in range(1, n_layers):
                self.add_module("lstm{}".format(n), LNPeepholeLSTMCUDA(hidden_size, hidden_size, True, dropout, dropout_on_output, eps))
            self.fc = nn.Linear(hidden_size, output_size)

            self.input_size = input_size
            self.hidden_size = hidden_size
            self.output_size = output_size
            self.n_layers = n_layers

        def forward(self, x, states):
            assert states[0].dim() == 3
            assert states[0].size(0) == self.n_layers
            assert states[0].size(1) == x.size(0)
            assert states[0].size(2) == self.hidden_size
            assert states[0].size() == states[1].size()

            new_hidden = torch.empty_like(states[0])
            new_cell = torch.empty_like(states[1])

            for n in range(self.n_layers):
                x, (new_hidden[n], new_cell[n]) = getattr(self, "lstm{}".format(n))(x, (states[0][n], states[1][n]))
            x = self.fc(x)

            return x, (new_hidden, new_cell)
    
    initialized[2] = True

### Instantiation
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [20]:
if not initialized[3]:
    device = ('cpu', 'cuda')[1]

    input_size = 5 #TEST 5
    hidden_size = 8 #TEST 8
    output_size = 6 #TEST 6
    n_layers = 3 #TEST 3
    dropout = 0. #TEST 0
    eps = 1e-05 #TEST 1e-05

    model_torch = LNPeepholeTorch(input_size, hidden_size, output_size, n_layers, dropout, eps)
    model_cpp = LNPeepholeCPP(input_size, hidden_size, output_size, n_layers, dropout, eps)
    model_cuda = LNPeepholeCUDA(input_size, hidden_size, output_size, n_layers, dropout, eps)

    model_torch.to(device)
    model_cpp.to(device)
    model_cuda.to(device)

    models = (model_torch, model_cpp, model_cuda)
    
    initialized[3] = True

### Parameter Synchronization
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [21]:
if not initialized[4]:
    named_parameter_dicts = [
        dict(model_torch.named_parameters()),
        dict(model_cpp.named_parameters()),
        dict(model_cuda.named_parameters())
    ]

    print("Synchronized Parameters:\n")
    for common_param_name in set.intersection(*(set(npd.keys()) for npd in named_parameter_dicts)):
        print("\t{}".format(common_param_name))
        for i in range(1, len(named_parameter_dicts)):
            if named_parameter_dicts[i][common_param_name].size() == named_parameter_dicts[0][common_param_name].size():
                named_parameter_dicts[i][common_param_name].data = named_parameter_dicts[0][common_param_name].data
            else:
                raise RuntimeError("Size mismatch\n0:{}\n{i}:{}".format(named_parameter_dicts[0][common_param_name].size(),
                                                                        named_parameter_dicts[i][common_param_name].size()))
    print()
    print("Exclusive Parameters (Not Synchronized):\n")
    for exclusive_param_name in set.union(*(set(npd.keys()) for npd in named_parameter_dicts)) - set.intersection(*(set(npd.keys()) for npd in named_parameter_dicts)):
        print("\t{}".format(exclusive_param_name))
        
    initialized[4] = True

---

## Creating a fake dataset
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [22]:
if not initialized[5]:
    def create_fake_loader(dataset_size, sequence_length, batch_size, drop_last=True):
        fake_inputs = torch.randn(dataset_size, sequence_length, input_size)
        fake_targets = torch.randint(high=output_size, size=(dataset_size, sequence_length), dtype=torch.int64)

        fake_dataset = TensorDataset(fake_inputs, fake_targets)

        fake_loader = DataLoader(fake_dataset, batch_size=batch_size, drop_last=drop_last)

        return fake_loader
    
    initialized[5] = True

In [23]:
if not initialized[6]:
    dataset_size = 1000
    sequence_length = 1 #TEST 20
    batch_size = 2 #TEST 8

    fake_loader = create_fake_loader(dataset_size, sequence_length, batch_size)
    print(next(iter(fake_loader))[0].size(), next(iter(fake_loader))[1].size())
    
    initialized[6] = True

---

## Sanity check: output comparison
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

### Forward Outputs
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [9]:
hidden = (torch.zeros(n_layers, batch_size, hidden_size, device=device), torch.zeros(n_layers, batch_size, hidden_size, device=device))

inputs = next(iter(fake_loader))[0].to(device)

for model in models:
    model.train()
#     model.eval()
del model # Removing temporary variable

In [10]:
with torch.no_grad():
    print("[model_torch]")
    print("\n{partial output}")
    out0 = model_torch(inputs, hidden)
    print(out0)

[model_torch]

{partial output}
(tensor([[[ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0

In [11]:
with torch.no_grad():
    print("[model_cpp]")
    print("\n{partial output}")
    out1 = model_cpp(inputs, hidden)
    print(out1)

[model_cpp]

{partial output}
(tensor([[[ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3

In [13]:
with torch.no_grad():
    print(out0[0].sub(out1[0]).abs().sum())

tensor(0., device='cuda:0')


In [14]:
with torch.no_grad():
    print("[model_cuda]")
    print("\n{partial output}")
    out2 = model_cuda(inputs, hidden)
    print(out2)

[model_cuda]

{partial output}
(tensor([[[ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.3354],
         [ 0.1514,  0.1963,  0.0375, -0.2102, -0.1708,  0.

In [15]:
with torch.no_grad():
    print(out0[0].sub(out2[0]).abs().sum())

tensor(0., device='cuda:0')


### Backward Gradients
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [24]:
criterion = nn.CrossEntropyLoss()

hidden = (torch.zeros(n_layers, batch_size, hidden_size, device=device, requires_grad=True), torch.zeros(n_layers, batch_size, hidden_size, device=device, requires_grad=True))

inputs, targets = next(iter(fake_loader))
inputs = inputs.to(device)
inputs.requires_grad_()
targets = targets.to(device)

inputs_grads = []
hidden_grads = []
for model in models:
    model.train()
#     model.eval()
    model.zero_grad()
    loss = criterion(model(inputs, hidden)[0].flatten(0, 1), targets.flatten(0, 1))
    print(loss)
    loss.backward()
    inputs_grads.append(inputs.grad.clone())
    inputs.grad.zero_()
    hidden_grads.append((hidden[0].grad.clone(), hidden[1].grad.clone()))
    hidden[0].grad.zero_()
    hidden[1].grad.zero_()

tensor(1.8271, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.8271, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.8271, device='cuda:0', grad_fn=<NllLossBackward>)


In [25]:
print("model_torch")
print(inputs_grads[0])
print(hidden_grads[0])
print(model_torch.lstm0.bias.grad)
print(model_torch.lstm0.gamma_cell.grad)
# print(model_torch.lstm0.weight_ih.grad[0 * hidden_size:1 * hidden_size])
# print(model_torch.lstm0.weight_ih.grad[1 * hidden_size:2 * hidden_size])
# print(model_torch.lstm0.weight_ih.grad[2 * hidden_size:3 * hidden_size])
# print(model_torch.lstm0.weight_ih.grad[3 * hidden_size:4 * hidden_size])

model_torch
tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0.,

In [26]:
print("model_cpp")
print(inputs_grads[1])
print(hidden_grads[1])
print(model_cpp.lstm0.bias.grad)
print(model_cpp.lstm0.gamma_cell.grad)
# print(model_cpp.lstm0.weight_ih.grad[0 * hidden_size:1 * hidden_size])
# print(model_cpp.lstm0.weight_ih.grad[1 * hidden_size:2 * hidden_size])
# print(model_cpp.lstm0.weight_ih.grad[2 * hidden_size:3 * hidden_size])
# print(model_cpp.lstm0.weight_ih.grad[3 * hidden_size:4 * hidden_size])

model_cpp
tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0

In [27]:
print(model_torch.lstm0.weight_ih.grad.sub(model_cpp.lstm0.weight_ih.grad).abs().sum())

tensor(0., device='cuda:0')


In [13]:
print("model_cuda")
print(inputs_grads[2])
print(hidden_grads[2])
print(model_cuda.lstm0.bias.grad)
print(model_cuda.lstm0.gamma_cell.grad)
# print(model_cuda.lstm0.weight_ih.grad[0 * hidden_size:1 * hidden_size])
# print(model_cuda.lstm0.weight_ih.grad[1 * hidden_size:2 * hidden_size])
# print(model_cuda.lstm0.weight_ih.grad[2 * hidden_size:3 * hidden_size])
# print(model_cuda.lstm0.weight_ih.grad[3 * hidden_size:4 * hidden_size])

model_cuda
tensor([[[-1.3435e+02, -9.0758e+01,  1.3010e+02, -1.8043e+02, -1.9627e+01],
         [ 6.4094e+02,  5.5853e+02, -1.1977e+03,  5.2183e+02, -1.3062e+03],
         [ 9.3651e-01, -8.1176e-01,  1.0687e+01,  1.0211e+01,  1.3905e+01],
         [ 1.9260e+01,  8.2067e+00,  2.1972e+01,  1.5270e+01,  4.9144e+01],
         [ 1.0636e+02,  2.7505e+02, -5.2485e+01,  3.3555e+02,  7.8988e+01],
         [ 7.6807e-01,  7.9251e-01, -1.4541e+00,  7.8489e-01,  4.9668e-01],
         [-6.6669e-03,  6.2959e+00, -1.2747e-01,  3.4855e+00,  1.3992e+00],
         [ 8.1981e+00,  6.3188e+00, -4.6396e+00,  3.2698e+00,  1.7502e+00],
         [-3.1055e+00, -1.2235e+00,  1.0010e+00, -4.7862e-01, -1.6232e+00],
         [-6.2164e+00, -1.6698e+00,  2.6211e+00, -6.9631e+00, -2.7720e+00],
         [ 4.7447e-01,  1.6326e+00, -3.8679e+00, -4.5445e-01, -9.3133e-01],
         [-8.3681e-01,  2.8994e-01, -1.3113e+00, -6.1282e-01, -2.1818e+00],
         [-1.7802e+00, -5.5835e+00,  1.6702e+00, -4.2400e+00, -9.6119e-01],
 

In [14]:
print(model_torch.lstm0.weight_ih.grad.sub(model_cuda.lstm0.weight_ih.grad).abs().sum())

tensor(35781.4258, device='cuda:0')


---

## Forward time comparison
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [37]:
dataset_size = 1000
sequence_length = 20
batch_size = 32

for model in models:
    model.train()
del model

fake_loader = create_fake_loader(dataset_size, sequence_length, batch_size, drop_last=True)

hidden = (torch.zeros(n_layers, batch_size, hidden_size, device=device), torch.zeros(n_layers, batch_size, hidden_size, device=device))

In [46]:
%%timeit -n 1 -r 20
with torch.no_grad():
    for inputs, _ in fake_loader:
        inputs = inputs.to(device)
        model_torch(inputs, hidden)

3.81 s ± 177 ms per loop (mean ± std. dev. of 20 runs, 1 loop each)


In [55]:
%%timeit -n 1 -r 20
with torch.no_grad():
    for inputs, _ in fake_loader:
        inputs = inputs.to(device)
        model_cpp(inputs, hidden)

5.55 s ± 66.8 ms per loop (mean ± std. dev. of 20 runs, 1 loop each)


In [56]:
%%timeit -n 1 -r 20
with torch.no_grad():
    for inputs, _ in fake_loader:
        inputs = inputs.to(device)
        model_cuda(inputs, hidden)

1.37 s ± 54.1 ms per loop (mean ± std. dev. of 20 runs, 1 loop each)


---

## +Backward time comparison
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [65]:
criterion = nn.CrossEntropyLoss()
"executed"

'executed'

In [74]:
%%timeit -r 20
for inputs, targets in fake_loader:
    inputs = inputs.to(device)
    targets = targets.to(device)
    model_torch.zero_grad()
    criterion(model_torch(inputs, hidden)[0].flatten(0, 1), targets.flatten(0, 1)).backward()

14.3 s ± 367 ms per loop (mean ± std. dev. of 20 runs, 1 loop each)


In [83]:
%%timeit -r 20
for inputs, targets in fake_loader:
    inputs = inputs.to(device)
    targets = targets.to(device)
    model_cpp.zero_grad()
    criterion(model_cpp(inputs, hidden)[0].flatten(0, 1), targets.flatten(0, 1)).backward()

9.94 s ± 307 ms per loop (mean ± std. dev. of 20 runs, 1 loop each)


In [84]:
%%timeit -r 20
for inputs, targets in fake_loader:
    inputs = inputs.to(device)
    targets = targets.to(device)
    model_cuda.zero_grad()
    criterion(model_cuda(inputs, hidden)[0].flatten(0, 1), targets.flatten(0, 1)).backward()

5.92 s ± 207 ms per loop (mean ± std. dev. of 20 runs, 1 loop each)


---